In [128]:
import matplotlib
import pathlib
import mne
import os
import numpy as np

In [129]:
# LOAD RAW:
matplotlib.use('Qt5Agg')

set_path = 'D:/raw_/original data/EXP/first time/resting-state-1019-1.edat3_013498_raw.set'
raw = mne.io.read_raw_eeglab(set_path,uint16_codec='latin1')
    # 上一行遇到问题了，记一下

Reading D:\raw_\original data\EXP\first time\resting-state-1019-1.edat3_013498_raw.fdt


In [130]:
# 为了计算后续crop长度&resampling目标频率，读取
freq = raw.info['sfreq']
n_samples = raw.n_times/freq
n_samples # 采样总数

750.0

In [133]:
# 参数修改后结果不如原来的，所以沿用最初参数
# 就算是同一组实验，不同样本间电极也有细微差别，在montage处报错后需要手动调整
# Delete non-eeg channels

non_eeg_chan = [
'GYR-X', 'GYR-Y', 'GYR-Z', 'ACC-X', 'ACC-Y', 'ACC-Z', 'MAG-X', 'MAG-Y', 'MAG-Z', 
    'SPO2', 'Pulse', 'HR', 'GSR','ECG', 'HEOR', 'HEOL', 'VEOU', 'VEOL','ECGM_27864797']

raw.drop_channels(non_eeg_chan)

raw.info['ch_names']
raw.plot()

Waiting for Loading-Thread to finish... (max. 10 sec)
Channels marked as bad:
none


In [134]:
# NO CHANNEL LOCATIONS IN EEG FILE

montage = mne.channels.make_standard_montage("standard_1005")
print(montage)

raw.set_montage(montage)
fig = raw.plot_sensors(show_names=True)

#定位完成

<DigMontage | 0 extras (headshape), 0 HPIs, 3 fiducials, 343 channels>


In [135]:
# Read events & crop
events, event_id = mne.events_from_annotations(raw)
# 源数据里没有语义化events，自己用词典
# event_id = {'Openeyes':events[1]['12'],'Closeeyes':events[1]['14']}


# 直接把tuple的描述字典赋值给event_id
# 每隔一秒建立一个event
event_id = 1  # 事件ID
duration = 1  # 事件持续时间/秒
events = mne.make_fixed_length_events(raw, event_id, start=0, duration=duration, overlap=0)
events

Used Annotations descriptions: ['12', '13', '14', '15']


array([[     0,      0,      1],
       [  1000,      0,      1],
       [  2000,      0,      1],
       ...,
       [747000,      0,      1],
       [748000,      0,      1],
       [749000,      0,      1]])

In [139]:
raw.plot()

Channels marked as bad:
['Fpz', 'Fp1', 'AF7', 'F8', 'FC6', 'CP1', 'T8']


In [140]:

print(raw.info['bads'])
raw.load_data()
raw.interpolate_bads()
raw.plot(title="after")

['Fpz', 'Fp1', 'AF7', 'F8', 'FC6', 'CP1', 'T8']
Setting channel interpolation method to {'eeg': 'spline'}.
Interpolating bad channels.
    Automatic origin fit: head of radius 95.2 mm
Computing interpolation matrix from 52 sensor positions
Interpolating 7 sensors


Channels marked as bad:
none


In [141]:
# resampling
target_sfreq = 500

raw_resampled = raw.copy().resample(sfreq=target_sfreq)
raw_resampled.plot_psd(fmax=250)

NOTE: plot_psd() is a legacy function. New code should use .compute_psd().plot().
Effective window size : 4.096 (s)


<MNELineFigure size 1000x350 with 2 Axes>

In [142]:
raw_resampled.plot()

Channels marked as bad:
none


In [143]:
# filtering
raw_filtered=raw_resampled.copy().filter(l_freq=1,h_freq=100)
raw_filtered
raw_filtered.plot()
raw_filtered.plot_psd(fmax=250)

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 1e+02 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 100.00 Hz
- Upper transition bandwidth: 25.00 Hz (-6 dB cutoff frequency: 112.50 Hz)
- Filter length: 1651 samples (3.302 s)



[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s


NOTE: plot_psd() is a legacy function. New code should use .compute_psd().plot().
Effective window size : 4.096 (s)


<MNELineFigure size 1000x350 with 2 Axes>

Channels marked as bad:
none


In [144]:
# ICA
# raw_ica = raw_filtered.copy()
# ica = mne.preprocessing.ICA(n_components= 20)

In [145]:
# ica.fit(raw_ica)
# raw_ica.load_data()
# ica.plot_sources(raw_ica)
# ica.plot_components()

In [146]:
# re-ref based on average channel(?
raw_ref = raw_filtered.copy()
raw_ref.load_data()
raw_ref.set_eeg_reference(ref_channels="average", projection=False)

EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.


Measurement date,Unknown
Experimenter,Unknown
Participant,Unknown
Digitized points,62 points
Good channels,59 EEG
Bad channels,None
EOG channels,Not available
ECG channels,Not available
Sampling frequency,500.00 Hz
Highpass,1.00 Hz
Lowpass,100.00 Hz


In [147]:
# 创建Epochs对象
reject_criteria = dict(eeg = 100e-6)

epochs = mne.Epochs(raw_ref,
                    events=events,
                    event_id=event_id,
                    tmin=-1,
                    tmax=1,
                    reject=reject_criteria,
                    preload=True)

# raw_cropped = raw.crop(tmin=t_start/1000,tmax=t_end/1000)
# raw_cropped.plot()

Not setting metadata
750 matching events found
Setting baseline interval to [-1.0, 0.0] s
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 750 events and 1001 original time points ...
    Rejecting  epoch based on EEG : ['Fpz', 'Fp1', 'Fp2', 'AF3', 'AF8', 'T8', 'TP8', 'P8', 'PO4', 'PO6', 'PO8', 'Oz', 'O1', 'O2']
    Rejecting  epoch based on EEG : ['Fpz', 'Fp1', 'Fp2', 'AF3', 'AF4', 'AF7', 'AF8', 'P8', 'O1', 'O2']
    Rejecting  epoch based on EEG : ['Fpz', 'Fp1', 'Fp2', 'AF3', 'AF4', 'AF7', 'AF8', 'F3', 'F5', 'F7', 'F8', 'FT8', 'C6', 'T8', 'TP8', 'P5', 'P7', 'P8', 'POz', 'PO3', 'PO4', 'PO5', 'PO6', 'PO7', 'PO8', 'Oz', 'O1', 'O2']
    Rejecting  epoch based on EEG : ['Fpz', 'Fp1', 'Fp2', 'AF3', 'AF4', 'AF7', 'AF8', 'F7', 'F8', 'FC6', 'FT8', 'C6', 'T8', 'CP5', 'TP7', 'TP8', 'P5', 'P7', 'P8', 'POz', 'PO3', 'PO4', 'PO5', 'PO6', 'PO7', 'Oz', 'O1', 'O2']
    Rejecting  epoch based on EEG : ['Fpz', 'Fp1', 'Fp2', 'AF4', 'AF7', 'AF8', 'F7

In [148]:
raw_ref.plot()

Channels marked as bad:
none


In [149]:
epochs.plot()

Dropped 0 epochs: 
The following epochs were marked as bad and are dropped:
[]
Channels marked as bad:
none


In [150]:
raw_ref.plot_psd()

NOTE: plot_psd() is a legacy function. New code should use .compute_psd().plot().
Effective window size : 4.096 (s)


<MNELineFigure size 1000x350 with 2 Axes>

In [151]:
save_dir='D:/raw_/results/exp/resting-state-1019-1.edat3_013498_raw.fif'
epochs.save(save_dir, overwrite=True)

C:\Users\Administrator\AppData\Local\Temp\ipykernel_12644\3556960661.py:2: RuntimeWarning: This filename (D:/raw_/results/exp/resting-state-1019-1.edat3_013498_raw.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  epochs.save(save_dir, overwrite=True)
